In [2]:
import os
import json
import gzip
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [4]:
import pickle
# Mounting Google drive to this file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Install JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [7]:
!tar -xvf spark-3.0.1-bin-hadoop2.7.tgz

spark-3.0.1-bin-hadoop2.7/
spark-3.0.1-bin-hadoop2.7/RELEASE
spark-3.0.1-bin-hadoop2.7/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/
spark-3.0.1-bin-hadoop2.7/examples/src/main/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/FPGrowthExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/GBTExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/ALSExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/KMeansExample.scala
spark-3.0.1-bin-hadoop2.7/example

In [8]:
!pip install -q findspark

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [10]:
# Create local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [12]:
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_metadata.csv')
# # total length of list, this number equals total number of products
print(len(data))

# # first row of the list
data['overview'].head(200)

45466


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0      Led by Woody, Andy's toys live happily in his ...
1      When siblings Judy and Peter discover an encha...
2      A family wedding reignites the ancient feud be...
3      Cheated on, mistreated and stepped on, the wom...
4      Just when George Banks has recovered from his ...
                             ...                        
195    Set in the year 1999 during the last days of t...
196    This simple romantic tragedy begins in 1957. G...
197    John Netherwood and his wife Leann are fugitiv...
198    While Jane Holman (Mastrantonio) is driving wi...
199    Young, wild poet Arthur Rimbaud and his mentor...
Name: overview, Length: 200, dtype: object

In [15]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [16]:
newdata = data.dropna(axis=0, subset=['overview'])
newdata=newdata[['original_title','overview']]
newdata

,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
45461,رگ خواب,Rising and falling between a man and woman.
45462,Siglo ng Pagluluwal,An artist struggles to finish his work while a...
45463,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [17]:
vectorizer = TfidfVectorizer(stop_words='english')
vec = vectorizer.fit_transform(newdata['overview'])
vec

<44512x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [18]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

!pip install -q findspark

import findspark
findspark.init()

In [19]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

sdf = spark.createDataFrame(newdata.astype(str))
sdf.show()
sdf.cache()

+--------------------+--------------------+
|      original_title|            overview|
+--------------------+--------------------+
|           Toy Story|Led by Woody, And...|
|             Jumanji|When siblings Jud...|
|    Grumpier Old Men|A family wedding ...|
|   Waiting to Exhale|Cheated on, mistr...|
|Father of the Bri...|Just when George ...|
|                Heat|Obsessive master ...|
|             Sabrina|An ugly duckling ...|
|        Tom and Huck|A mischievous you...|
|        Sudden Death|International act...|
|           GoldenEye|James Bond must u...|
|The American Pres...|Widowed U.S. pres...|
|Dracula: Dead and...|When a lawyer sho...|
|               Balto|An outcast half-w...|
|               Nixon|An all-star cast ...|
|    Cutthroat Island|Morgan Adams and ...|
|              Casino|The life of the g...|
|Sense and Sensibi...|Rich Mr. Dashwood...|
|          Four Rooms|It's Ted the Bell...|
|Ace Ventura: When...|Summoned from an ...|
|         Money Train|A vengeful

DataFrame[original_title: string, overview: string]

In [20]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover
from pyspark.ml.evaluation import ClusteringEvaluator
from pandas.core.reshape.concat import concat

tokenizer = Tokenizer(inputCol="overview", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="stopWordsRemovedTokens")
hashingTF = HashingTF(inputCol="stopWordsRemovedTokens", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

In [21]:
kmeans = KMeans(k=12)
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, kmeans])

In [22]:
model = pipeline.fit(sdf)

In [23]:
results = model.transform(sdf)

In [24]:
results.show()

+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+
|      original_title|            overview|              tokens|stopWordsRemovedTokens|         rawFeatures|            features|prediction|
+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+
|           Toy Story|Led by Woody, And...|[led, by, woody,,...|  [led, woody,, and...|(2000,[87,189,295...|(2000,[87,189,295...|         0|
|             Jumanji|When siblings Jud...|[when, siblings, ...|  [siblings, judy, ...|(2000,[0,72,170,2...|(2000,[0,72,170,2...|         0|
|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|  [family, wedding,...|(2000,[66,104,129...|(2000,[66,104,129...|         0|
|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|  [cheated, on,, mi...|(2000,[86,111,417...|(2000,[86,111,417...|         0|
|Father of th

In [25]:
sdf1 = results.select("original_title","prediction")
sdf2 = sdf1.toPandas()
sdf2 = sdf2.groupby(['prediction'])['original_title'].apply(lambda x: ','.join(x)).reset_index()
sdf2 = sdf2.rename(columns={"prediction": "Cluster Number", "original_title": "Movie name"})
sdf2

,Cluster Number,Movie name
0,0,"Toy Story,Jumanji,Grumpier Old Men,Waiting to ..."
1,1,"Sudden Death,Get Shorty,Assassins,Persuasion,D..."
2,2,"Москва,Il signor Robinson, mostruosa storia d'..."
3,3,"La machine,Sappho,Miss Lulu Bett,ఒక్కడు"
4,4,"A Month by the Lake,Hedd Wyn,The Lady Vanishes..."
5,5,"Höstsonaten,Mermaids,Milka - elokuva tabuista,..."
6,6,"Ricomincio da tre,Mer om Oss Barn i Bullerbyn,..."
7,7,FC Venus
8,8,"Sonic Outlaws,The Net,Three Wishes,Even Cowgir..."
9,9,Courage


In [26]:
sdf2.to_csv(r'/content/drive/My Drive/recom.csv')
sdf2 = pd.read_csv('/content/drive/My Drive/recom.csv')

In [27]:
def search(a):
  l = [(a, )]
  gh = spark.createDataFrame(l, ['overview'])
  pred = model.transform(gh)
  output = pred.collect()[0]['prediction']
  fin = sdf2.loc[sdf2['Cluster Number'] == output]
  fin1 = fin['Movie name'].values[0]
  fin1 = fin1.split(',')
  return fin1

In [28]:
def unique(a):
  uni = []
  for i in a:
    if i not in uni:
      uni.append(i)
  return uni

In [29]:
a = input("Enter movie names to find similar movies: ")
search(a)
unique(search(a))[:20]

Enter movie names to find similar movies: Heat


['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Copycat']